# Препроцессор для данных

In [1]:
import pandas as pd
import numpy as np

from pathlib import Path
import joblib

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

In [2]:
RAW_DIR = Path("../data/raw")

pq_path  = RAW_DIR / "orange_belgium_std.parquet"

df = pd.read_parquet(pq_path)  

print("| shape:", df.shape) 
df.head(3)                                   

| shape: (11896, 180)


,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,FACTOR11,FACTOR12,FACTOR13,FACTOR14,FACTOR15,FACTOR16,FACTOR17,FACTOR18,treatment,churn
0,-3.064007,-1.272259,-4.075844,1.492514,2.328039,-0.330240,-0.770389,0.035541,2.439325,-0.847266,...,V2,V2,V5,V20,V9,V11,V1,V4,0,0
1,-4.574066,-3.541815,1.107371,0.447314,-0.471140,-0.567309,0.195963,0.383654,1.523472,-0.184596,...,V2,V3,V6,V20,V9,V15,V4,V4,0,0
2,-1.751471,-2.039692,-3.788823,0.624226,0.564614,-0.713385,1.003502,1.582819,-1.461687,0.844975,...,V5,V2,V11,V20,V9,V1,V4,V4,1,0


In [3]:
y = df["churn"].astype(int)

X = df.drop(columns=["churn", "treatment"])

In [4]:
# Константы: если у признака <2 уникальных значений (все одинаковые/почти одинаковые) — выкидываем
const_cols = [c for c in X.columns if X[c].nunique(dropna=False) < 2]
X = X.drop(columns=const_cols)

print(f"удалено констант: {len(const_cols)}")

удалено констант: 1


In [5]:
# Сначала отделим test 15% от ВСЕХ данных
X_tmp, X_test, y_tmp, y_test = train_test_split(
    X, y,
    test_size=0.15,          # 15% пойдут в финальный тест
    stratify=y,              # стратификация по y — сохраняем долю класса 1 (важно при 3.4% churn)
    random_state=42
)

# Потом отделим валидацию от оставшегося 
valid_rel = 0.15 / 0.85      # чтобы в сумме val ≈ 15% исходных
X_train, X_valid, y_train, y_valid = train_test_split(
    X_tmp, y_tmp,
    test_size=valid_rel,     
    stratify=y_tmp,          
    random_state=42
)

X_train.shape, X_valid.shape, X_test.shape

((8326, 177), (1785, 177), (1785, 177))

In [6]:
num_cols = X_train.select_dtypes(include="number").columns.tolist()

cat_cols = [c for c in X_train.columns if c not in num_cols]

print(f"numeric: {len(num_cols)}, categorical: {len(cat_cols)}")

numeric: 160, categorical: 17


In [7]:
# Для числовых: скейлим до "среднее=0, std=1"
num_pipe = Pipeline(steps=[
    ("scaler",  StandardScaler())                   # чтобы модели не страдали от разных шкал
])

# Для категориальных: кодируем в OHE
cat_pipe = Pipeline(steps=[
    ("ohe",     OneHotEncoder(handle_unknown="ignore"))       # игнорим новые уровни на вал/тесте
])

# Склеиваем обе ветки: к числовым применится num_pipe, к категориальным — cat_pipe
preprocessor = ColumnTransformer(transformers=[
    ("num", num_pipe, num_cols),
    ("cat", cat_pipe, cat_cols)
])

In [8]:
# fit_transform на train — "учим"
X_train_proc = preprocessor.fit_transform(X_train)

# transform на valid/test — применяем уже "выученное" к новым данным (без переобучения)
X_valid_proc = preprocessor.transform(X_valid)
X_test_proc  = preprocessor.transform(X_test)

# Проверяем размеры матриц. После OHE колонок станет больше.
X_train_proc.shape, X_valid_proc.shape, X_test_proc.shape


((8326, 335), (1785, 335), (1785, 335))

In [9]:
PROC_DIR = Path("../data/processed")
PROC_DIR.mkdir(parents=True, exist_ok=True)

# Сохраняем препроцессор
joblib.dump(preprocessor, PROC_DIR / "preprocessor.pkl")

# Целевые вектора пригодятся для обучения/оценки
np.save(PROC_DIR / "y_train.npy", y_train.to_numpy())
np.save(PROC_DIR / "y_valid.npy", y_valid.to_numpy())
np.save(PROC_DIR / "y_test.npy",  y_test.to_numpy())
